In [7]:
# setup
import os
import requests
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor  # registers the pandas .spatial accessor
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
# from utils import *
# local path
out_dir = Path("Data")
# api url
derq_api_url = 'https://api-external.cloud.derq.com'
# Set the API key as an environment variable
headers = {
    "x-api-key": os.getenv('derq-api-key')
}
# global variables
all_event_types = 'IC, WWD, STPV, TV, LCV, RLV, NM-VV, NM-VRU, CRSH'
# default_speed_buckets = '5,10,15,20,25'

In [8]:
# get feature service as dataframe
def get_fs_as_df(url: str):
    layer = FeatureLayer(url, gis=GIS()) 
    return pd.DataFrame.spatial.from_layer(layer)


In [9]:
# get data from TRPA services
smart_location_url = "https://maps.trpa.org/server/rest/services/Transportation_SMART/FeatureServer/0"
smart_safety_url   = "https://maps.trpa.org/server/rest/services/Transportation_SMART/FeatureServer/1"
smart_speed_url    = "https://maps.trpa.org/server/rest/services/Transportation_SMART/FeatureServer/2"
smart_traffic_url  = "https://maps.trpa.org/server/rest/services/Transportation_SMART/FeatureServer/3"
smart_vru_url      = "https://maps.trpa.org/server/rest/services/Transportation_SMART/FeatureServer/4"
smart_daily_url    = "https://maps.trpa.org/server/rest/services/Transportation_SMART/FeatureServer/5"

# data frrame from url
df_location = get_fs_as_df(smart_location_url)
df_safety   = get_fs_as_df(smart_safety_url)
df_speed    = get_fs_as_df(smart_speed_url)
df_traffic  = get_fs_as_df(smart_traffic_url)
df_vru      = get_fs_as_df(smart_vru_url)
df_daily    = get_fs_as_df(smart_daily_url)


c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.trpa.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.trpa.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.trpa.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.

In [10]:
# your existing dataframes
dfs = {
    "SMART_Location": df_location,
    "SMART_Safety": df_safety,
    "SMART_Speed": df_speed,
    "SMART_Traffic": df_traffic,
    "SMART_VRU": df_vru,
    "SMART_Daily": df_daily
}

schema_rows = []

for name, df in dfs.items():
    for col, dtype in df.dtypes.items():
        schema_rows.append({
            "file_name": name,
            "variable": col,
            "datatype": str(dtype),
            "description": ""  # placeholder – you can fill manually if desired
        })

schema_df = pd.DataFrame(schema_rows)

# write to CSV
schema_df.to_csv("trpa_smart_schema.csv", index=False)
print(schema_df.head(12).to_string(index=False))


     file_name         variable       datatype description
SMART_Location         OBJECTID          Int64            
SMART_Location       LocationId         string            
SMART_Location     LocationName         string            
SMART_Location         Latitude        Float64            
SMART_Location        Longitude        Float64            
SMART_Location         GlobalID         string            
SMART_Location     created_user         string            
SMART_Location     created_date datetime64[ns]            
SMART_Location last_edited_user         string            
SMART_Location last_edited_date datetime64[ns]            
SMART_Location           Status         string            
SMART_Location        DateRange         string            


In [ ]:
df_safety.head()

In [ ]:
# get max and min date from safety data
max_date = df_safety['TimeAtSite'].max()
min_date = df_safety['TimeAtSite'].min()
# built date range text
date_range_text = f"{min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}"
print(f"Data available from {date_range_text}")